# Introduction

This notebook explains how to clean cosmic void catalogues to extract cosmological constraints from void statistics. 

The only required inputs are the positions of the void centres and of the tracers used to detect the voids (and possibly their masses). 

With this notebook you will remove spurious voids, rescale their radii to a specific density threshold, and reject voids in case of overlaps. Then you will measure the void size function and compare it to theoretical predictions. The latter are computed with a re-parametrisation of the volume-conserving model (Vdn), that takes into account the effect of the tracer bias (see Ronconi et al. 2019, Contarini et al. 2019).

# Set the inputs

To ensure compatibility in Python versions 2.x and 3.x

In [ ]:
from __future__ import print_function

Import the CBL functions and system modules:

In [ ]:
import CosmoBolognaLib as cbl

import time
import sys
import os

To show the standard output stream of the CBL functions (that are implemented in C++ and converted to python via SWIG) on notebook cells (instead of on terminal) the package wurlitzer has to be used (see https://github.com/minrk/wurlitzer)

In [ ]:
%load_ext wurlitzer

The paramters are read from the parameter file:

In [ ]:
filename = "../input/parameter_file.ini"
print ("Loading parameters from", filename)
param = cbl.ReadParameters(filename)

The coordinate type is selected (warning: observed coordinates are still not implemented!):

In [ ]:
if param.findBool('comovingCoordinates') :
    coordinates = cbl.CoordinateType__comoving_
else :
    coordinates = cbl.CoordinateType__observed_

# Load the input void catalogue
Only the void centre coordinates have to be provided to construct the input void catalogue, while all the other necessary attributes will be computed automatically

In [ ]:
cast = []
clmn = []
attrNames = ['X_coord', 'Y_coord', 'Z_coord', 'Radius', 'centralDensity', 'densityContrast']
attrAv = [cbl.Var__X_, cbl.Var__Y_, cbl.Var__Z_, cbl.Var__Radius_, cbl.Var__CentralDensity_,\
          cbl.Var__DensityContrast_]
for ii in range(len(attrNames)) :
    if param.findBool(attrNames[ii]) :
        cast.append(attrAv[ii])
        clmn.append(param.findInt(attrNames[ii]+'_clmn'))
clmn, cast = (list(x) for x in zip(*sorted(zip(clmn, cast))))

attr = cbl.VarCast(cast)

A new void catalogue is created, reading the input file and sorting the attributes according the selected order:

In [ ]:
vdcat = cbl.Catalogue(cbl.ObjectType__Void_,
                       coordinates,
                       attr,
                       clmn,
                       [param.findString('inputVoidCatalogue')],
                       param.findInt('vd_comments'))

The main properties of the catalogue (volume, density and mean interparticle separation of the sample) are computed using the lenght of the catalogue side. For the current implementation, the catalogue geometry has to be a box. If the boxside is not provided, it will be computed using the maximum separaration between the tracers on the x-axis.

In [ ]:
if (param.findDouble('boxside') < 0.) :
    boxside = abs(vdcat.Max(cbl.Var__X_) - vdcat.Min(cbl.Var__X_))
else :
    boxside = param.findDouble('boxside')

# Load the input tracer catalogue

The tracer catalogue can be either a Gadget-2.0 or an ASCII file. Using the parameters 'fact' and 'nSub', it is possible to convert the distance units (e.g. fact = 0.001 for converting kpc/h to Mpc/h) and to sub-sample the objects of the catalogue, respectively. <br> Only for ASCII catalogues, a mass factor and/or a mass cut-off can be applied to the tracer catalogue (obviously only if the tracer mass is provided). The first one can be activated with the parameter 'Munit', that represents the mass units in units of solar masses. The cut-off is applied by selecting a value larger than $0$ for the minimum mass of the catalogue, given by the 'Mmin' parameter.
In the end, the main properties of the catalogue are computed using (once again) the lenght of the boxside.

In [ ]:
if param.findBool('Gadget') :
    if not param.findBool('comovingCoordinates') :
        ErrorCBL('Observed coordinates not available for Gadget snapshot.')
    else :
        trcat = cbl.Catalogue (cbl.ObjectType__Halo_,
                           param.findString('inputTracersFile'),
                           param.findBool('swapEndianism'),
                           param.findDouble('fact'),
                           True,
                           param.findDouble('nSub'))
else :
    if param.findBool('comovingCoordinates') :
        tr_cast = []
        tr_clmn = []
        trAttrNames = ['X_coord_tr', 'Y_coord_tr', 'Z_coord_tr', 'Mass']
        trAttrAv = [cbl.Var__X_, cbl.Var__Y_, cbl.Var__Z_, cbl.Var__Mass_]
        for ii in range(len(trAttrNames)) :
            if param.findBool(trAttrNames[ii]) :
                tr_cast.append(trAttrAv[ii])
                tr_clmn.append(param.findInt(trAttrNames[ii]+'_clmn'))        
        tr_clmn, tr_cast = (list(x) for x in zip(*sorted(zip(tr_clmn, tr_cast))))  # orders clmn and cast according to column order 
        tr_attr = cbl.VarCast(tr_cast)

        temp = cbl.Catalogue (cbl.ObjectType__Halo_,
                              coordinates,
                              tr_attr,
                              tr_clmn,
                              [param.findString('inputTracersFile')],
                              param.findInt('tr_comments'),
                              param.findDouble('nSub'),
                              param.findDouble('fact'))

        if not param.findBool('Mass') :
            trcat = temp
            temp = None
            print ("Finished reading input tracers catalogue.")

        else :
            print ("Finished reading input tracers catalogue, now applying mass scale factor and/or cut-off ... ")

            # scale factor
            if (param.findDouble('Munit') > 0.) :
                for ii in range(temp.nObjects()) :
                    mass = temp.mass(ii)*param.findDouble('Munit')
                    temp.set_var(ii, cbl.Var__Mass_, mass)

            # mass cut-off
            if (param.findDouble('Mmin') > 0.) :
                trcat = cbl.Catalogue()
                trcat = temp.sub_catalogue(cbl.Var__Mass_, param.findDouble('Mmin'), temp.Max(cbl.Var__Mass_), False)
            else :
                trcat = temp
                temp = None

        print ("\t ... done!")

  # observed coordinates
    else :
        print ("Observed coordinates not supported with this configuration...")
        exit(1)

If the void radii are not read from the input void catalogue, they are temporarily set to the maximum value of the range of the accepted radii:

In [ ]:
if not param.findBool('Radius') :
    delta_r = param.findVectorDouble('delta_r')
    radii = [delta_r[1] for ii in range(vdcat.nObjects())]
    vdcat.set_var(cbl.Var__Radius_, radii)

A 3-dimensional chain-mesh for the input tracer catalogue is generated. The cell size of the chain-mesh is equal to 2 times the value of the mean interparticle separation of the tracer catalogue. The maximum radius of the chain-mesh is given by the maximum value of the radius of the voids in the catalogue.

In [ ]:
ChM = cbl.ChainMesh3D (2.*trcat.mps(),
                       trcat.var(cbl.Var__X_),
                       trcat.var(cbl.Var__Y_),
                       trcat.var(cbl.Var__Z_),
                       vdcat.Max(cbl.Var__Radius_))

The central density and the density contrast are computed if they are not read from the input void catalogue. <br> The central density (in units of the average density) is computed as the density of a sphere centred in the void centre and with radius $R = ratio \cdot R_\mathrm{eff}$, where $R_\mathrm{eff}$ is the void effective radius and $ratio$ is a parameter $<1$ selected by the user. The density contrast is the ratio between the central density and the density within the sphere centred in the void centre and with radius $R = R_\mathrm{eff}$. <br> With the compute_densityContrast function, the effect of cloud-in-void is taken into account and the voids with central density larger than the density at $r_{eff}$ are rejected.

The criterion for the overlap-check is read from the parameter file. In case of overlap: <br>
1) if ol_crit = false $\rightarrow$ the void with the higher central density is rejected, <br>
2) if ol_crit = true $\rightarrow$ the void with the lower density constrast is rejected.

In [ ]:
ol_crit = cbl.Var__DensityContrast_ if param.findInt('ol_crit') == 1 else cbl.Var__CentralDensity_

The threshold is the value of the spherically-averaged density contrast ($\rho_m/\,\overline{\rho}+1$) that each void will contain after the rescaling procedure. In this example, the threshold is not rescaled for different redshifts. This choise is not unique: the prescription is effective as long as the same density contrast is mantained for both the cleaning procedure and the theoretical void size function model.

In [ ]:
threshold = param.findDouble('deltav_NL')+1.

# Costruct the cleaned catalogue
To build the final cleaned void catalogue, you can select different procedures to perform using the following parameters: <br>
 - clean1 = true $\rightarrow$ erase voids with underdensities higher than a given threshold<br>
 - clean2 = true $\rightarrow$ erase voids with effective radii outside a given range  <br>
 - clean3 = true $\rightarrow$ erase voids with density contrast lower than a given value, specified by the 'relevance' parameter <br>
 - delta_r $\rightarrow$ range of acceptable radii, voids with radii outside this range are erased <br>
 - rescale = true $\rightarrow$ the rescaling procedure will be performed: firstly the algorithm checks if within an initial radius the enclosed density is higher or lower than the selected density threshold; then it shrinks or expands the initial radius to match the required density threshold <br>
 - overlap = true $\rightarrow$  overlapping voids are erased from the catalogue: when two voids do overlap one of them is erased according to the chosen overlap criterion.

In [ ]:
vdcat.clean_void_catalogue( param.findBool('initial_radius'),
                            param.findVectorDouble('delta_r'),
                            threshold,
                            param.findBool('rescale'),
                            trcat,
                            ChM,
                            param.findDouble('ratio'),
                            param.findBool('overlap'),
                            ol_crit)

In the end, the cleaned catalogue is stored in an ASCII file (if the output directory does not exist, it will be created).

In [ ]:
clmnsToPrint = cbl.VarCast(attrAv)

if not os.path.exists(param.findString('outputDir')):
    os.makedirs(param.findString('outputDir'))

vdcat.write_data(param.findString('outputDir')+param.findString('VoidCatalogueFile'), clmnsToPrint)

# Compute the void size function
Once the cleaned void catalogue is built, the size function can be computed in order to compare the data to the theoretical predictions given by the Vdn model.

First, you need to import the required CBL functions and the matplotlib packages.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import csv
from matplotlib.pyplot import figure
%matplotlib inline

The cleaned void catalogue is loaded and the peripheral voids (close to the boundaries of the simulation box) are rejected, as their radii are not accurately rescaled by our cleaning algorithm, (periodic boundary conditions have not been implemented yet).

In [ ]:
cast = []
clmn = []
attrNames = ['X_coord', 'Y_coord', 'Z_coord', 'Radius', 'centralDensity', 'densityContrast']
attrAv = [cbl.Var__X_, cbl.Var__Y_, cbl.Var__Z_, cbl.Var__Radius_, cbl.Var__CentralDensity_, cbl.Var__DensityContrast_]
for ii in range(len(attrNames)) :
    if param.findBool(attrNames[ii]) :
        cast.append(attrAv[ii])
        clmn.append(param.findInt(attrNames[ii]+'_clmn'))
clmn, cast = (list(x) for x in zip(*sorted(zip(clmn, cast))))  # orders clmn and cast according to column order 

attr = cbl.VarCast(cast)

inputVoidCatalogue = param.findString('outputDir')+param.findString('VoidCatalogueFile')

print (" Input void catalogue: ", inputVoidCatalogue)

vdcat = cbl.Catalogue (cbl.ObjectType__Void_,
                       coordinates,
                       attr,
                       clmn,
                       [inputVoidCatalogue],
                       param.findInt('vd_comments'))

cut = param.findDouble('cut')
xmin = trcat.Min(cbl.Var__X_)
ymin = trcat.Min(cbl.Var__Y_)
zmin = trcat.Min(cbl.Var__Z_)
xmax = trcat.Max(cbl.Var__X_)
ymax = trcat.Max(cbl.Var__Y_)
zmax = trcat.Max(cbl.Var__Z_)

i=0
while(i<vdcat.nObjects()) :
    if (vdcat.xx(i)+vdcat.radius(i) > xmax-cut) : vdcat.remove_object(i)
    elif (vdcat.yy(i)+vdcat.radius(i) > ymax-cut) : vdcat.remove_object(i)
    elif (vdcat.zz(i)+vdcat.radius(i) > zmax-cut) : vdcat.remove_object(i)
    elif (vdcat.xx(i)-vdcat.radius(i) < xmin+cut) : vdcat.remove_object(i)
    elif (vdcat.yy(i)-vdcat.radius(i) < ymin+cut) : vdcat.remove_object(i)
    elif (vdcat.zz(i)-vdcat.radius(i) < zmin+cut) : vdcat.remove_object(i)
    else : i=i+1

if (param.findDouble('boxside') < 0.) :
    boxside = (xmax-xmin)-cut*2
else :
    boxside = param.findDouble('boxside')-cut*2.

According to its definition, the void size funciton can be computed as the comoving number density of voids in logarithmic bins of effective radii, divided by the logarithmic extent of the bins. Here the void abundance is measured with this prescription and saved as a text file. 

In [ ]:
NC = cbl.NumberCounts1D_Size(vdcat, param.findInt("nbin"), param.findDouble("rmin"), param.findDouble("rmax"),
                             param.findDouble("shift"), cbl.HistogramType__dn_dlnV_,vdcat.volume())
NC.measure(cbl.ErrorType__Poisson_)
NC.write(param.findString("outputDir"), "Void_size_distr_z%4.2f"%(param.findDouble("redshift"))+".out")

rrp, ptp, erp = NC.dataset().xx(), NC.dataset().data(), NC.dataset().error()

A cosmological model is defined, using the cosmological parameters provided in the parameter file.

In [ ]:
cosm = cbl.Cosmology(param.findDouble('OmM'),
                     param.findDouble('Omb'),
                     param.findDouble('Omn'),
                     param.findDouble('massless'),
                     param.findInt('massive'),
                     param.findDouble('OmL'),
                     param.findDouble('Omr'),
                     param.findDouble('hh'),
                     param.findDouble('As'),
                     param.findDouble('pivot'),
                     param.findDouble('ns'),
                     param.findDouble('w0'),
                     param.findDouble('wa'),
                     param.findDouble('fNL'),
                     param.findInt('type_NG'),
                     param.findDouble('tau'),
                     param.findString('model'),
                     param.findBool('unit'))

cosm.set_sigma8(param.findDouble('sigma8'))

The redshift of the catalogue is read from the parameter file.

In [ ]:
zz = param.findDouble('redshift')
print (" Redshift =", zz)

The effective bias of the mass tracer catalogue has been computed previously by the modelling of large-scale two-point correlation function.

In [ ]:
b_eff = param.findDouble('b_eff')
print (" Effective bias =", b_eff)

The effective bias has to be converted to recover the one that characterises the relation between the tracer and the dark matter distribution inside cosmic voids. To do that, a pre-calibrated linear relation is used (see Contarini et al. 2019).

In [ ]:
slope = param.findDouble('slope')
offset = param.findDouble('offset')

Non-linear underdensity threshold used to compute the theoretical model.

In [ ]:
del_v = param.findDouble('deltav_NL')
print (" Non-linear underdensity threshold =", del_v)

The theoretical void size function is computed at different radii and is rescaled to take into account the effect of the tracer bias.

In [ ]:
rrt = np.linspace(param.findDouble('xmin'), param.findDouble('xmax'), param.findInt('ngrid'))
sff_s = cosm.size_function(rrt, zz, "Vdn", b_eff, 1., 0., del_v)
sff_r = cosm.size_function(rrt, zz, "Vdn", b_eff, slope, offset, del_v)

The results are plotted to compare the measured void abundances with the theoretical predictions.

In [ ]:
figure(num=None, figsize=(6, 6), dpi=100, facecolor='w', edgecolor='k')

ax = plt.gca()

ax.set_xscale('log')
ax.set_yscale('log')

ax.xaxis.set_ticks([2.e+1, 2.5e+1, 3.e+1, 3.5e+1, 4.e+1, 4.5e+1 ,5.e+1, 5.5e+1, 6.e+1, 6.5e+1, 7.e+1])
ax.minorticks_off()
ax.xaxis.set_ticklabels(['$20$', '$25$', '$30$', '$35$', '$40$', '$45$', '$50$', '$55$', '$60$', '$65$', '$70$'])
ax.yaxis.set_ticks([1.e-11, 1.e-10, 1.e-9, 1.e-8, 1.e-7, 1.e-6, 1.e-5, 1.e-4])
ax.yaxis.set_ticklabels(['$-11$', '$-10$', '$-9$', '$-8$', '$-7$', '$-6$', '$-5$', '$-4$'])
ax.grid(zorder=0, color='lightgray', alpha=0.3)
ax.set_xlabel("$R_\mathrm{eff}\\ [h^{-1} \ \mathrm{Mpc}]$", fontsize=16)
ax.set_ylabel("$\log (\mathrm{d}n/\mathrm{d ln} {R_\mathrm{eff}})\\ [h^3 \ \mathrm{Mpc}^{-3}]$", fontsize=16)
ax.get_yaxis().set_tick_params(which='both', direction='in', labelsize=14)
ax.get_xaxis().set_tick_params(which='both', direction='in', labelsize=14)

ax.plot(rrp, ptp, "or", label = "Void counts", markersize=7, color="gold", markeredgecolor='black', markeredgewidth=1.3, zorder=3)
ax.errorbar(rrp, ptp, yerr=erp, fmt='none', ecolor='k', elinewidth=1.5, zorder=3)

ax.plot(rrt, sff_s, color='orangered', linewidth=2, ls='--', alpha=0.8, label = "Standard Vdn model")
ax.plot(rrt, sff_r, color='mediumseagreen', linewidth=2, ls='-', alpha=0.8, label = "Re-parametrised Vdn model")

ptp = [ptp[ii] for ii in range(len(ptp)) if ptp[ii]>0]

ax.set_ylim([min(ptp)*0.1, max(ptp)*12])
ax.set_xlim([param.findDouble('xmin'), param.findDouble('xmax')])

ax.legend(loc='best', fontsize=14, numpoints=1)